In [ ]:
# USA Black-Scholes Pricing on USO ETF Options

import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from datetime import datetime

# Load USO options data
df = pd.read_csv("../data/uso_options_data.csv")
df = df.dropna(subset=["impliedVolatility"])

# Filter out-of-the-money call options only
df = df[(df["inTheMoney"] == False) & (df["strike"] > 50)]
df = df.head(50).copy()

# Assume USO current price manually or fetch from yfinance
S = 67.0
r = 0.05  # annual risk-free rate

# Calculate time to expiration in years
def get_T(expiration_date):
    exp_date = datetime.strptime(expiration_date, "%Y-%m-%d")
    delta = (exp_date - datetime.today()).days
    return max(delta / 365, 1 / 365)

df["T"] = df["expirationDate"].apply(get_T)

# Black-Scholes function
def black_scholes(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

# Calculate theoretical prices
df["bs_price"] = df.apply(
    lambda row: black_scholes(S, row["strike"], row["T"], r, row["impliedVolatility"]),
    axis=1
)

# Compute error
df["abs_error"] = np.abs(df["bs_price"] - df["lastPrice"])

# Plot market vs model
plt.figure(figsize=(10, 6))
plt.plot(df["strike"], df["bs_price"], l_]()
